In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [3]:
df=pd.read_csv("ball_by_ball_it20.csv")
df.head()

,Unnamed: 0,Match ID,Date,Venue,Bat First,Bat Second,Innings,Over,Ball,Batter,...,Winner,Chased Successfully,Total Batter Runs,Total Non Striker Runs,Batter Balls Faced,Non Striker Balls Faced,Player Out Runs,Player Out Balls Faced,Bowler Runs Conceded,Valid Ball
0,0,1339605,2023-03-26,SuperSport Park,West Indies,South Africa,1,1,1,BA King,...,South Africa,1,1,0,1,0,NaN,NaN,1,1
1,1,1339605,2023-03-26,SuperSport Park,West Indies,South Africa,1,1,2,KR Mayers,...,South Africa,1,1,1,1,1,NaN,NaN,1,1
2,2,1339605,2023-03-26,SuperSport Park,West Indies,South Africa,1,1,3,BA King,...,South Africa,1,0,1,0,1,1.0,2.0,0,1
3,3,1339605,2023-03-26,SuperSport Park,West Indies,South Africa,1,1,4,J Charles,...,South Africa,1,0,1,1,1,NaN,NaN,0,1
4,4,1339605,2023-03-26,SuperSport Park,West Indies,South Africa,1,1,5,J Charles,...,South Africa,1,4,1,2,1,NaN,NaN,4,1


In [4]:
df.columns

Index(['Unnamed: 0', 'Match ID', 'Date', 'Venue', 'Bat First', 'Bat Second',
       'Innings', 'Over', 'Ball', 'Batter', 'Non Striker', 'Bowler',
       'Batter Runs', 'Extra Runs', 'Runs From Ball', 'Ball Rebowled',
       'Extra Type', 'Wicket', 'Method', 'Player Out', 'Innings Runs',
       'Innings Wickets', 'Target Score', 'Runs to Get', 'Balls Remaining',
       'Winner', 'Chased Successfully', 'Total Batter Runs',
       'Total Non Striker Runs', 'Batter Balls Faced',
       'Non Striker Balls Faced', 'Player Out Runs', 'Player Out Balls Faced',
       'Bowler Runs Conceded', 'Valid Ball'],
      dtype='object')

In [5]:
df["Bat First"].unique()

array(['West Indies', 'Scotland', 'Estonia', 'Portugal', 'Kuwait',
       'Sri Lanka', 'Swaziland', 'Belize', 'England',
       'United Arab Emirates', 'Denmark', 'Bangladesh', 'Finland',
       'India', 'Papua New Guinea', 'Afghanistan', 'Japan', 'Croatia',
       'Netherlands', 'Ireland', 'New Zealand', 'Romania', 'Pakistan',
       'Gibraltar', 'Rwanda', 'Malaysia', 'Malta', 'Cyprus', 'Nepal',
       'Austria', 'Samoa', 'Nigeria', 'South Africa', 'Australia',
       'Belgium', 'Oman', 'Bahamas', 'Seychelles', 'Panama', 'Jersey',
       'Zimbabwe', 'Hong Kong', 'Kenya', 'China', 'Uganda', 'Botswana',
       'Ghana', 'Vanuatu', 'Cayman Islands', 'Bahrain', 'Malawi',
       'Isle of Man', 'Cameroon', 'Norway', 'Singapore', 'Maldives',
       'Hungary', 'Canada', 'St Helena', 'Tanzania', 'Serbia', 'Namibia',
       'Philippines', 'Germany', 'Bulgaria', 'Thailand', 'Lesotho',
       'Czech Republic', 'Luxembourg', 'Spain', 'Mali', 'Bhutan',
       'Slovenia', 'Indonesia', 'Argentina', 'B

In [6]:
valid_countries=['West Indies','Sri Lanka','England','Bangladesh','India', 'Afghanistan','Netherlands', 'Ireland', 'New Zealand','Pakistan','Nepal','South Africa', 'Australia','Zimbabwe']
df=df[df["Bat First"].isin(valid_countries)]
df=df[df["Bat Second"].isin(valid_countries)]
df

,Unnamed: 0,Match ID,Date,Venue,Bat First,Bat Second,Innings,Over,Ball,Batter,...,Winner,Chased Successfully,Total Batter Runs,Total Non Striker Runs,Batter Balls Faced,Non Striker Balls Faced,Player Out Runs,Player Out Balls Faced,Bowler Runs Conceded,Valid Ball
0,0,1339605,2023-03-26,SuperSport Park,West Indies,South Africa,1,1,1,BA King,...,South Africa,1,1,0,1,0,NaN,NaN,1,1
1,1,1339605,2023-03-26,SuperSport Park,West Indies,South Africa,1,1,2,KR Mayers,...,South Africa,1,1,1,1,1,NaN,NaN,1,1
2,2,1339605,2023-03-26,SuperSport Park,West Indies,South Africa,1,1,3,BA King,...,South Africa,1,0,1,0,1,1.0,2.0,0,1
3,3,1339605,2023-03-26,SuperSport Park,West Indies,South Africa,1,1,4,J Charles,...,South Africa,1,0,1,1,1,NaN,NaN,0,1
4,4,1339605,2023-03-26,SuperSport Park,West Indies,South Africa,1,1,5,J Charles,...,South Africa,1,4,1,2,1,NaN,NaN,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425114,425114,1187667,2019-11-05,Saxton Oval,New Zealand,England,2,20,2,TK Curran,...,New Zealand,0,11,2,9,2,NaN,NaN,0,1
425115,425115,1187667,2019-11-05,Saxton Oval,New Zealand,England,2,20,3,TK Curran,...,New Zealand,0,12,2,10,2,NaN,NaN,1,1
425116,425116,1187667,2019-11-05,Saxton Oval,New Zealand,England,2,20,4,S Mahmood,...,New Zealand,0,2,12,3,10,NaN,NaN,0,1
425117,425117,1187667,2019-11-05,Saxton Oval,New Zealand,England,2,20,5,S Mahmood,...,New Zealand,0,3,12,4,10,NaN,NaN,1,1


In [7]:
second_innings=df[df["Innings"]==2]
df2=second_innings.groupby(["Match ID","Over","Ball"]).sum()[['Runs From Ball','Wicket']].reset_index()
target=second_innings[["Match ID",'Target Score']]
target.drop_duplicates(inplace=True)
int(target[target["Match ID"]==211028]["Target Score"])


180

In [8]:
df2["current score"]=df2.groupby("Match ID").cumsum()["Runs From Ball"]
df2["wickets"]=df2.groupby("Match ID").cumsum()["Wicket"]
df2.drop(columns=["Runs From Ball","Wicket"],inplace=True)
df2

,Match ID,Over,Ball,current score,wickets
0,211028,1,1,0,0
1,211028,1,2,4,0
2,211028,1,3,4,0
3,211028,1,4,4,0
4,211028,1,5,8,0
...,...,...,...,...,...
98026,1384637,20,2,148,7
98027,1384637,20,3,148,7
98028,1384637,20,4,148,8
98029,1384637,20,5,148,8


In [9]:
df2=df2.merge(target,on="Match ID",how="left")
df2


,Match ID,Over,Ball,current score,wickets,Target Score
0,211028,1,1,0,0,180
1,211028,1,2,4,0,180
2,211028,1,3,4,0,180
3,211028,1,4,4,0,180
4,211028,1,5,8,0,180
...,...,...,...,...,...,...
98026,1384637,20,2,148,7,186
98027,1384637,20,3,148,7,186
98028,1384637,20,4,148,8,186
98029,1384637,20,5,148,8,186


In [10]:
df2["runs left"]=df2["Target Score"]-df2["current score"]
df2["balls left"]=120-((df2["Over"]-1)*6+df2["Ball"])
df2["wickets left"]=10-df2["wickets"]
df2["crr"]=round(df2["current score"]*6/((df2["Over"]-1)*6+df2["Ball"]),2)
df2.drop(columns=["wickets","current score"],inplace=True)
df2

,Match ID,Over,Ball,Target Score,runs left,balls left,wickets left,crr
0,211028,1,1,180,180,119,10,0.00
1,211028,1,2,180,176,118,10,12.00
2,211028,1,3,180,176,117,10,8.00
3,211028,1,4,180,176,116,10,6.00
4,211028,1,5,180,172,115,10,9.60
...,...,...,...,...,...,...,...,...
98026,1384637,20,2,186,38,4,3,7.66
98027,1384637,20,3,186,38,3,3,7.59
98028,1384637,20,4,186,38,2,2,7.53
98029,1384637,20,5,186,38,1,2,7.46


In [11]:
df2=df2[df2["balls left"]!=0]
df2["rrr"]=round(df2["runs left"]*6/df2["balls left"],2)
df2

,Match ID,Over,Ball,Target Score,runs left,balls left,wickets left,crr,rrr
0,211028,1,1,180,180,119,10,0.00,9.08
1,211028,1,2,180,176,118,10,12.00,8.95
2,211028,1,3,180,176,117,10,8.00,9.03
3,211028,1,4,180,176,116,10,6.00,9.10
4,211028,1,5,180,172,115,10,9.60,8.97
...,...,...,...,...,...,...,...,...,...
98025,1384637,20,1,186,38,5,3,7.72,45.60
98026,1384637,20,2,186,38,4,3,7.66,57.00
98027,1384637,20,3,186,38,3,3,7.59,76.00
98028,1384637,20,4,186,38,2,2,7.53,114.00


In [12]:
match_details=df[['Match ID', 'Date', 'Venue', 'Bat First', 'Bat Second']]
match_details.drop_duplicates(inplace=True)
match_details

,Match ID,Date,Venue,Bat First,Bat Second
0,1339605,2023-03-26,SuperSport Park,West Indies,South Africa
1151,356009,2009-06-14,Lord's,Sri Lanka,Ireland
1750,1072317,2018-02-07,Bellerive Oval,England,Australia
2482,533283,2012-09-25,Pallekele International Cricket Stadium,Bangladesh,Pakistan
2940,1262759,2021-07-28,R Premadasa Stadium,India,Sri Lanka
...,...,...,...,...,...
423368,1119543,2018-07-03,Old Trafford,England,India
424188,1298178,2022-11-10,Adelaide Oval,India,England
424410,426724,2009-11-13,Dubai International Cricket Stadium,Pakistan,New Zealand
424660,682919,2014-03-21,Sylhet Stadium,Ireland,Netherlands


In [13]:
df2=match_details.merge(df2,on="Match ID",how="right")
df2

,Match ID,Date,Venue,Bat First,Bat Second,Over,Ball,Target Score,runs left,balls left,wickets left,crr,rrr
0,211028,2005-06-13,The Rose Bowl,England,Australia,1,1,180,180,119,10,0.00,9.08
1,211028,2005-06-13,The Rose Bowl,England,Australia,1,2,180,176,118,10,12.00,8.95
2,211028,2005-06-13,The Rose Bowl,England,Australia,1,3,180,176,117,10,8.00,9.03
3,211028,2005-06-13,The Rose Bowl,England,Australia,1,4,180,176,116,10,6.00,9.10
4,211028,2005-06-13,The Rose Bowl,England,Australia,1,5,180,172,115,10,9.60,8.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97732,1384637,2023-08-20,Malahide,India,Ireland,20,1,186,38,5,3,7.72,45.60
97733,1384637,2023-08-20,Malahide,India,Ireland,20,2,186,38,4,3,7.66,57.00
97734,1384637,2023-08-20,Malahide,India,Ireland,20,3,186,38,3,3,7.59,76.00
97735,1384637,2023-08-20,Malahide,India,Ireland,20,4,186,38,2,2,7.53,114.00


In [14]:
res=df[["Match ID",'Chased Successfully']]
res.drop_duplicates(inplace=True)
df2=df2.merge(res,on="Match ID",how="left")
df2

,Match ID,Date,Venue,Bat First,Bat Second,Over,Ball,Target Score,runs left,balls left,wickets left,crr,rrr,Chased Successfully
0,211028,2005-06-13,The Rose Bowl,England,Australia,1,1,180,180,119,10,0.00,9.08,0
1,211028,2005-06-13,The Rose Bowl,England,Australia,1,2,180,176,118,10,12.00,8.95,0
2,211028,2005-06-13,The Rose Bowl,England,Australia,1,3,180,176,117,10,8.00,9.03,0
3,211028,2005-06-13,The Rose Bowl,England,Australia,1,4,180,176,116,10,6.00,9.10,0
4,211028,2005-06-13,The Rose Bowl,England,Australia,1,5,180,172,115,10,9.60,8.97,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97732,1384637,2023-08-20,Malahide,India,Ireland,20,1,186,38,5,3,7.72,45.60,0
97733,1384637,2023-08-20,Malahide,India,Ireland,20,2,186,38,4,3,7.66,57.00,0
97734,1384637,2023-08-20,Malahide,India,Ireland,20,3,186,38,3,3,7.59,76.00,0
97735,1384637,2023-08-20,Malahide,India,Ireland,20,4,186,38,2,2,7.53,114.00,0


In [15]:
pure=df2.drop(columns=[ 'Over', 'Ball','Target Score',"Match ID",'Date'])
pure.rename(columns={"Chased Successfully":"result"},inplace=True)
pure

,Venue,Bat First,Bat Second,runs left,balls left,wickets left,crr,rrr,result
0,The Rose Bowl,England,Australia,180,119,10,0.00,9.08,0
1,The Rose Bowl,England,Australia,176,118,10,12.00,8.95,0
2,The Rose Bowl,England,Australia,176,117,10,8.00,9.03,0
3,The Rose Bowl,England,Australia,176,116,10,6.00,9.10,0
4,The Rose Bowl,England,Australia,172,115,10,9.60,8.97,0
...,...,...,...,...,...,...,...,...,...
97732,Malahide,India,Ireland,38,5,3,7.72,45.60,0
97733,Malahide,India,Ireland,38,4,3,7.66,57.00,0
97734,Malahide,India,Ireland,38,3,3,7.59,76.00,0
97735,Malahide,India,Ireland,38,2,2,7.53,114.00,0


In [16]:
pure.to_csv("model training dataset.csv")

In [17]:
x=pure.iloc[:,:-1]
y=pure.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.8)
x.describe()


,runs left,balls left,wickets left,crr,rrr
count,97737.000000,97737.000000,97737.00000,97737.000000,97737.000000
mean,92.280661,64.286504,7.26469,7.335667,10.443846
std,49.656606,32.806349,2.30095,2.451207,13.907903
min,-18.000000,1.000000,-1.00000,0.000000,-24.000000
25%,53.000000,37.000000,6.00000,6.000000,6.740000
50%,91.000000,65.000000,8.00000,7.250000,8.710000
75%,128.000000,93.000000,9.00000,8.620000,10.970000
max,273.000000,119.000000,10.00000,60.000000,822.000000


In [18]:
ct=ColumnTransformer([("trf",OneHotEncoder(sparse=False,drop="first"),['Venue','Bat First', 'Bat Second'])],remainder="passthrough")


In [19]:
pipeline_rf=Pipeline(steps=[("step 1",ct),("step 2",RandomForestClassifier())])
pipeline_rf.fit(x_train,y_train)

Pipeline(steps=[('step 1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['Venue', 'Bat First',
                                                   'Bat Second'])])),
                ('step 2', RandomForestClassifier())])

In [20]:
pipeline_lr=Pipeline(steps=[("step 1",ct),("step 2",LogisticRegression(solver="liblinear"))])
pipeline_lr.fit(x_train,y_train)
y_pred_lr=pipeline_lr.predict(x_test)
print("Accuracy with Logistic regression:",accuracy_score(y_test,y_pred_lr))
for i in range(20):
    print(pipeline_lr.predict_proba(x_test)[i])

Accuracy with Logistic regression: 0.8602123033636014
[0.01451375 0.98548625]
[0.93733575 0.06266425]
[0.91118591 0.08881409]
[0.98887474 0.01112526]
[0.64969143 0.35030857]
[0.90135398 0.09864602]
[0.9146296 0.0853704]
[0.0311565 0.9688435]
[0.9718809 0.0281191]
[0.10068968 0.89931032]
[0.0789029 0.9210971]
[0.10766044 0.89233956]
[0.18311711 0.81688289]
[0.73247356 0.26752644]
[0.36614824 0.63385176]
[0.14135526 0.85864474]
[0.17078463 0.82921537]
[0.08371291 0.91628709]
[0.77704459 0.22295541]
[0.77849393 0.22150607]


In [21]:
# trf_x_train=ct.fit_transform(x_train)
# trf_x_test=ct.fit_transform(x_test)
# params={"C":[0.01,0.1,1,10],"kernel":["linear","sigmoid","poly","rbf"],"degree":[2,3,4]}
# grid=GridSearchCV(SVC(),params)
# grid.fit(trf_x_train,y_train)

In [22]:
# print(grid.best_estimator_)
# print(grid.score(trf_x_test,y_test))

In [23]:
# pipeline_svc=Pipeline(steps=[("step 1",ct),("step 2",SVC(C=10, degree=2, kernel='linear',probability=True))])
# pipeline_svc.fit(x_train,y_train)
# y_pred_svc=pipeline_svc.predict(x_test)
# print("Accuracy with suppor vector classifier:",accuracy_score(y_test,y_pred_svc))

In [24]:
# for i in range(20):
#     print(pipeline_svc.predict_proba(x_test)[i])

In [25]:
# rf=RandomForestClassifier(n_estimators=1000)
# rf.fit(trf_x_train, y_train)
# y_pred=rf.predict(trf_x_test)
# print(accuracy_score(y_test,y_pred))
# for i in range(20):
#     print(rf.predict_proba(trf_x_test)[i])

In [26]:
pipeline_rf=Pipeline(steps=[("step 1",ct),("step 2",RandomForestClassifier())])
pipeline_rf.fit(x_train,y_train)
y_pred_rf=pipeline_lr.predict(x_test)
print("Accuracy with Random Forest:",accuracy_score(y_test,y_pred_lr))
for i in range(20):
    print(pipeline_rf.predict_proba(x_test)[i])

Accuracy with Random Forest: 0.8602123033636014
[0. 1.]
[0.92 0.08]
[0.88 0.12]
[0.96 0.04]
[0.51 0.49]
[0.62 0.38]
[0.67 0.33]
[0.01 0.99]
[1. 0.]
[0. 1.]
[0.09 0.91]
[0.08 0.92]
[0.92 0.08]
[0.13 0.87]
[0.16 0.84]
[0.08 0.92]
[0.16 0.84]
[0.1 0.9]
[0.38 0.62]
[0.75 0.25]


In [27]:
prog_df=df.groupby(["Match ID","Innings","Over","Ball"]).sum()[['Runs From Ball',"Wicket"]].reset_index()
prog_df["current score"]=prog_df.groupby("Match ID").cumsum()["Runs From Ball"]
prog_df

,Match ID,Innings,Over,Ball,Runs From Ball,Wicket,current score
0,211028,1,1,1,0,0,0
1,211028,1,1,2,1,0,1
2,211028,1,1,3,0,0,1
3,211028,1,1,4,0,0,1
4,211028,1,1,5,0,0,1
...,...,...,...,...,...,...,...
204973,1384637,2,20,2,0,0,333
204974,1384637,2,20,3,0,0,333
204975,1384637,2,20,4,0,1,333
204976,1384637,2,20,5,0,0,333


In [28]:
innings_2=prog_df[prog_df["Innings"]==2]
innings_1=prog_df[prog_df["Innings"]==1]
innings_1
temp=innings_2.groupby("Match ID").count()["Ball"].reset_index()

In [29]:
innings_1.to_csv("first_innings.csv")
innings_2.to_csv("second_innings.csv")

In [30]:
def innings_progression(match):
    match_df_2=innings_2[innings_2["Match ID"]==match]
    match_df_1=innings_1[innings_1["Match ID"]==match]
    target=match_df_2.iloc[0]["current score"]
    match_df_2["Current score"]=match_df_2["current score"]-target
    match_df_2.drop(columns=["current score"],inplace=True)
    match_df_2.rename(columns={"Current score":"current score"},inplace=True)
    match_df_2["balls"]=(match_df_2["Over"]-1)*6+match_df_2["Ball"]
    match_df_2.drop(columns=["Over","Ball","Innings"],inplace=True)
    match_df_1["balls"]=(match_df_1["Over"]-1)*6+match_df_1["Ball"]
    match_df_1.drop(columns=["Over","Ball","Innings"],inplace=True)
    wickets_1=match_df_1[match_df_1["Wicket"]==1]
    wickets_2=match_df_2[match_df_2["Wicket"]==1]
    innings_1_runs=match_df_1["current score"].to_list()
    innings_1_wickets_x=wickets_1["balls"].to_list()
    innings_1_wickets_y=wickets_1["current score"].to_list()
    innings_2_wickets_x=wickets_2["balls"].to_list()
    innings_2_wickets_y=wickets_2["current score"].to_list()
    innings_1_balls=match_df_1["balls"].to_list()
    innings_2_runs=match_df_2["current score"].to_list()
    innings_2_balls=match_df_2["balls"].to_list()
    fig=go.Figure()
    fig.add_trace(go.Line(x=innings_1_balls,y=innings_1_runs,name="first innings"))
    fig.add_trace(go.Line(x=innings_2_balls,y=innings_2_runs,name="second innings"))
    fig.add_trace(go.Scatter(x=innings_1_wickets_x,y=innings_1_wickets_y,mode="markers",marker_color="blue",name="first innings wickets"))
    fig.add_trace(go.Scatter(x=innings_2_wickets_x,y=innings_2_wickets_y,mode="markers",marker_color="red",name="second innings wickets"))
    fig.update_layout(width=800,height=400,title="Innings progression",
    xaxis_title="Balls",
    yaxis_title="runs",)
    fig.show()
innings_progression(1339605)

c:\python ssss\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [31]:
def match_progression(match):
    match_df_2=innings_2[innings_2["Match ID"]==match]
    target=match_df_2.iloc[0]["current score"]
    match_df_2["Current score"]=match_df_2["current score"]-target
    match_df_2.drop(columns=["current score"],inplace=True)
    match_df_2.rename(columns={"Current score":"current score"},inplace=True)
    match_df_2["balls"]=(match_df_2["Over"]-1)*6+match_df_2["Ball"]
    match_df_2.drop(columns=["Ball","Innings"],inplace=True)
    match_df=df2.drop(columns=['Ball','Target Score','Date'])
    match_df=match_df[match_df["Match ID"]==match]
    columns=match_df.columns.to_list()
    overs_df=pd.DataFrame(columns=columns)
    initial=match_df.iloc[0].values.tolist()
    initial[4]=0
    initial[6]=120
    initial[7]=10
    overs_df.loc[0]=initial
    cnt=1
    for i in range(5,len(match_df),6):
        overs_df.loc[cnt]=match_df.iloc[i].values.tolist()
        cnt+=1
    over_prog=match_df_2.groupby("Over").sum()[["Runs From Ball","Wicket"]].reset_index()
    runs_per_over=over_prog["Runs From Ball"].to_list()
    overs=overs_df["Over"].to_list()
    wickets=over_prog["Wicket"].to_list()
    wickets=wickets[:-1]
    wickets.insert(0,0)
    overs_df.drop(columns=["Match ID","Chased Successfully","Over"],inplace=True)
    win,lose=[],[]
    columns=overs_df.columns
    for i in range(len(overs_df)):
        record=pd.DataFrame(overs_df.iloc[i])
        values=[[rec[0] for rec in record.values]]
        x=pd.DataFrame(values,columns=columns)
        probabilities=pipeline_lr.predict_proba(x)[0]
        win.append(round(probabilities[1]*100))
        lose.append(round(probabilities[0]*100))
    fig=go.Figure()
    fig.add_trace(go.Bar(x=overs,y=runs_per_over,name="runs per over"))
    fig.add_trace(go.Scatter(x=overs,y=win,mode="markers+lines",marker_color="green",name="win  probability"))
    fig.add_trace(go.Scatter(x=overs,y=lose,mode="markers+lines",marker_color="red",name="loss probability"))
    fig.add_trace(go.Scatter(x=overs,y=wickets,mode="markers+lines",marker_color="black",name="wickets per over"))
    fig.show()
match_progression(1339605)


In [32]:
date="2023-03-26"
batting_first="West Indies"
chasing="South Africa"
venue="SuperSport Park"
df[(df["Date"]==date) & (df["Bat First"]==batting_first) & (df["Bat Second"]==chasing) & (df["Venue"]==venue)]["Match ID"].unique()
# df[(df["Date"]==date)]

array([1339605], dtype=int64)

In [33]:
df2.to_csv("pure.csv")

In [ ]:
pickle